In [4]:
%load_ext autoreload
%autoreload 2

import os
import sys
import torch

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import torch
import torch_geometric.transforms as T
from generator import RoadNetwork, Trajectory

from evaluation.tasks import (
    NextLocationPrediciton, DestinationPrediciton, RoutePlanning
)

from models import PCAModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
network = RoadNetwork()
network.load("../../osm_data/porto")
trajectory = Trajectory(
    "../../datasets/trajectories/Porto/road_segment_map_final.csv", nrows=100000
)
traj_dataset = trajectory.generate_TTE_datatset()
data = network.generate_road_segment_pyg_dataset()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

In [31]:
from sklearn import metrics 
nextlocation_pred = NextLocationPrediciton(
        traj_dataset=traj_dataset,
        network=network,
        device=device,
        batch_size=256,
        epochs=20,
        seed=88,
    )
nextlocation_pred.register_metric(
    name="accuracy",
    metric_func=metrics.accuracy_score,
    args={"normalize": True},
)

Pandas Apply:   0%|          | 0/80000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/80000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20000 [00:00<?, ?it/s]

In [29]:
model = PCAModel(data, device)
emb = model.load_emb()

In [11]:
def masked_softmax(x, idxs):
    mask = torch.zeros_like(x)
    for row, idx in zip(mask, idxs):
        row[idx] = 1
    divider = torch.sum(torch.exp(x)*mask, axis=1).reshape(-1, 1)
    mask = mask + 1e-44
    return torch.log(torch.div(torch.exp(x)*mask, divider))

soft = torch.nn.Softmax(dim=1)
e = torch.tensor([[0,0,.5,0.5], [.3,0.4,0.2,0.2], [.2,0.4,0.1,0.3]])
print(masked_softmax(e, [[0,1,2],[2,3], [1]]))
soft(e)

tensor([[  -1.2944,   -1.2944,   -0.7944, -102.1803],
        [-101.8926, -101.8926,   -0.6931,   -0.6931],
        [-101.4872,    0.0000, -101.6695, -101.4872]])


tensor([[0.1888, 0.1888, 0.3112, 0.3112],
        [0.2554, 0.2823, 0.2311, 0.2311],
        [0.2363, 0.2887, 0.2138, 0.2612]])

In [32]:
nextlocation_pred.evaluate(emb)

Average training loss in episode 0: 109.24584405186077
Average training loss in episode 1: 108.06136407334203
Average training loss in episode 2: 105.29422550384228
Average training loss in episode 3: 100.63558438334603
Average training loss in episode 4: 94.23842501411804
Average training loss in episode 5: 86.65973851018059
Average training loss in episode 6: 79.05279411827794
Average training loss in episode 7: 70.8377650812411
Average training loss in episode 8: 63.66940263742075
Average training loss in episode 9: 57.51557824863032
Average training loss in episode 10: 52.55236369123855
Average training loss in episode 11: 48.186794439443766
Average training loss in episode 12: 44.55339768138556
Average training loss in episode 13: 41.65924235578543
Average training loss in episode 14: 38.845493487275824
Average training loss in episode 15: 36.29170030222152
Average training loss in episode 16: 34.21508403936514
Average training loss in episode 17: 32.281467657119705
Average traini

{'accuracy': 0.504}

In [10]:
# Test Destination Prediction
from sklearn import metrics 
destination_pred = DestinationPrediciton(
        traj_dataset=traj_dataset,
        network=network,
        device=device,
        batch_size=256,
        epochs=20,
        seed=88,
    )
destination_pred.register_metric(
    name="accuracy",
    metric_func=metrics.accuracy_score,
    args={"normalize": True},
)

Pandas Apply:   0%|          | 0/80000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/80000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20000 [00:00<?, ?it/s]

In [4]:
model = PCAModel(data, device)
emb = model.load_emb()

In [11]:
destination_pred.evaluate(emb)

Average training loss in episode 0: 8.002796031415652
Average training loss in episode 1: 7.728944015198241
Average training loss in episode 2: 7.618062226536175
Average training loss in episode 3: 7.447741648640496
Average training loss in episode 4: 7.154178783916437
Average training loss in episode 5: 6.751626476312217
Average training loss in episode 6: 6.273732733802674
Average training loss in episode 7: 5.801170958497654
Average training loss in episode 8: 5.36764315827586
Average training loss in episode 9: 4.973850305088031
Average training loss in episode 10: 4.623563499877247
Average training loss in episode 11: 4.311056073862143
Average training loss in episode 12: 4.0435809243601355
Average training loss in episode 13: 3.8067675817507904
Average training loss in episode 14: 3.5993725583195304
Average training loss in episode 15: 3.4162518094522882
Average training loss in episode 16: 3.2540437459183957
Average training loss in episode 17: 3.113460920108393
Average training

{'accuracy': 0.2001}

In [46]:
# Test RoutePlanning
from sklearn import metrics 
route_pred = RoutePlanning(
        traj_dataset=traj_dataset,
        network=network,
        device=device,
        batch_size=256,
        epochs=20,
        seed=88,
    )
route_pred.register_metric(
    name="accuracy",
    metric_func=metrics.accuracy_score,
    args={"normalize": True},
)

Pandas Apply:   0%|          | 0/80000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/80000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20000 [00:00<?, ?it/s]

In [7]:
model = PCAModel(data, device)
emb = model.load_emb()

In [47]:
route_pred.evaluate(emb)

Average training loss in episode 0: 223.33841259258622
Average training loss in episode 1: 220.99884233078637
Average training loss in episode 2: 219.19950505948296
Average training loss in episode 3: 216.19173497209152
Average training loss in episode 4: 212.5699733941319
Average training loss in episode 5: 208.74989304222618
Average training loss in episode 6: 205.81459035660131
Average training loss in episode 7: 203.30335998535156
Average training loss in episode 8: 201.20129340906112
Average training loss in episode 9: 199.44695196669704
Average training loss in episode 10: 197.66103955351124
Average training loss in episode 11: 196.11294984665162
Average training loss in episode 12: 195.00290545868796
Average training loss in episode 13: 193.93958928409856
Average training loss in episode 14: 192.9389164836262
Average training loss in episode 15: 192.13443865791297
Average training loss in episode 16: 191.4252798062163
Average training loss in episode 17: 190.77533027234549
Avera

{'accuracy': 0.008468326516222565}